# <div style="text-align: center; color: #1a5276;">Custom Loss</div>

## <font color='blue'>  Table of Contents </font>

1. [Introduction](#1)
2. [Setup](#2)
3. [Example 1](#3)
4. [Example 2](#4)
5. [Example 3](#5)
6. [References](#references)

<a name="1"></a>
## <font color='blue'> 1. Introduction </font>

This notebook demonstrates how to create custom loss functions in PyTorch using simple examples. Custom losses allow you to tailor model optimization to specific goals, which can be crucial for tasks like imbalanced data, specific error penalization, or unique performance metrics. We'll cover the basics and provide practical examples for better understanding.

<a name="2"></a>
## <font color='blue'> 2. Setup </font>

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
torch.manual_seed(42)

<a name="3"></a>
## <font color='blue'> 3. Example 1 </font>

This simplified example defines a custom MSE loss function and compares it with PyTorch's built-in MSE loss to confirm they produce the same result. 

In [5]:
class CustomMSELoss(nn.Module):
    """
    Custom Mean Squared Error (MSE) Loss.

    This custom loss function computes the mean squared error between 
    predicted values (y_pred) and true values (y_true).

    Usage:
        loss_fn = CustomMSELoss()
        loss = loss_fn(y_pred, y_true)
    """
    def __init__(self):
        super(CustomMSELoss, self).__init__()  # Initialize the parent class (nn.Module).

    def forward(self, y_pred, y_true):
        """
        Computes the mean squared error loss.

        Args:
            y_pred (torch.Tensor): Predicted values.
            y_true (torch.Tensor): Ground truth values.

        Returns:
            torch.Tensor: The calculated MSE loss.
        """
        return torch.mean((y_pred - y_true) ** 2)  # Element-wise squared difference, then mean.


In [7]:
# Example usage
y_pred = torch.randn(5, 1, requires_grad=True)
y_true = torch.randn(5, 1)

# Custom MSE
custom_mse = CustomMSELoss()(y_pred, y_true)

custom_mse

tensor(0.2877, grad_fn=<MeanBackward0>)

In [ ]:
Now we will see if we get the same results using the PyTorch built-in function.

In [8]:
# PyTorch built-in MSE
builtin_mse = F.mse_loss(y_pred, y_true)

print(f"Custom MSE: {custom_mse.item():.6f}")
print(f"Built-in MSE: {builtin_mse.item():.6f}")

Custom MSE: 0.287668
Built-in MSE: 0.287668


As expected, we get the same results.

<a name="4"></a>
## <font color='blue'> 4. Example 2 </font>

This custom loss function combines binary cross-entropy and mean squared error, with a weighting factor alpha to balance the two. 

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomLoss(nn.Module):
    """
    Custom Loss combining Binary Cross-Entropy (BCE) and Mean Squared Error (MSE).

    This loss function is useful when balancing between classification (BCE) 
    and regression (MSE) objectives. The `alpha` parameter controls the trade-off:
      - `alpha = 1` focuses entirely on BCE.
      - `alpha = 0` focuses entirely on MSE.

    Usage:
        loss_fn = CustomLoss(alpha=0.7)
        loss = loss_fn(y_pred, y_true)
    """
    def __init__(self, alpha=0.5):
        """
        Initializes the CustomLoss class.

        Args:
            alpha (float): Weight for the BCE loss term (0 ≤ alpha ≤ 1).
        """
        super(CustomLoss, self).__init__()  # Initialize the parent class (nn.Module).
        self.alpha = alpha  # Store the alpha value for balancing the losses.

    def forward(self, y_pred, y_true):
        """
        Computes the weighted combination of BCE and MSE losses.

        Args:
            y_pred (torch.Tensor): Predicted values.
            y_true (torch.Tensor): Ground truth values.

        Returns:
            torch.Tensor: The combined loss value.
        """
        # Compute Binary Cross-Entropy loss
        bce_loss = F.binary_cross_entropy(y_pred, y_true)
        
        # Compute Mean Squared Error loss
        mse_loss = F.mse_loss(y_pred, y_true)
        
        # Combine the losses using alpha for weighting
        return self.alpha * bce_loss + (1 - self.alpha) * mse_loss

    
# Example usage
y_pred = torch.sigmoid(torch.randn(5, 1, requires_grad=True))
y_true = torch.randint(0, 2, (5, 1)).float()

criterion = CustomLoss(alpha=0.7)
loss = criterion(y_pred, y_true)
print(f"Loss: {loss.item()}")

Loss: 0.5575313568115234


<a name="5"></a>
## <font color='blue'> 5. Example 3 </font>

In this example the goal is to demonstrate the integration of a custom loss function in a training loop.

In [11]:
# Define a simple custom MSE loss function
class CustomMSELoss(nn.Module):
    def __init__(self):
        super(CustomMSELoss, self).__init__()

    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)  # One input feature, 10 output features
        self.fc2 = nn.Linear(10, 1)  # Output a single value

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create a random dataset
x_train = torch.randn(100, 1)  # 100 data points, 1 feature
y_train = 3 * x_train + 2 + torch.randn_like(x_train) * 0.5  # Linear relation with some noise

# Initialize the model, optimizer, and loss function
model = SimpleNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = CustomMSELoss()

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_train)

    # Compute the loss
    loss = loss_fn(y_pred, y_train)

    # Zero the gradients, backward pass, optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 11.3141
Epoch [20/100], Loss: 8.0045
Epoch [30/100], Loss: 4.4206
Epoch [40/100], Loss: 2.0854
Epoch [50/100], Loss: 1.0460
Epoch [60/100], Loss: 0.5868
Epoch [70/100], Loss: 0.3805
Epoch [80/100], Loss: 0.2915
Epoch [90/100], Loss: 0.2548
Epoch [100/100], Loss: 0.2403


**Explanation:**

- Custom Loss Function: CustomMSELoss is defined to compute the Mean Squared Error (MSE) between predicted values (y_pred) and actual values (y_true).


- Model: A simple feedforward neural network (SimpleNN) with one hidden layer and ReLU activation.


- Training Loop:

    - The model makes predictions (y_pred).

    - The loss is calculated using the custom loss function (loss_fn).

    - Gradients are computed with loss.backward(), and the model parameters are updated with optimizer.step().

This example demonstrates how to incorporate a custom loss function in a basic PyTorch training loop.

<a name="references"></a>
## <font color='blue'> References </font>

[PyTorch Documentation](https://pytorch.org/docs/stable/index.html)